# New Section

In [0]:
import functools
import sys
from functools import reduce
from functools import partial
import queue
import re
import operator
import time

In [0]:
ops = {'+': operator.add,
       '-': operator.sub,
       '*': operator.mul,
       '/': operator.truediv,
       '==': operator.eq,
       '!=': operator.ne,
       '<': operator.lt,
       '<=': operator.le,
       '>': operator.gt,
       '>=': operator.ge,
       'abs': operator.abs,
       '^': operator.pow
       }
def foc(x, value):
    return x == value


class ConstraintVar:
    def __init__(self, d, n):
        self.domain = [v for v in d]
        self.name = n


class UnaryConstraint:
    def __init__(self, v, fn):
        self.var = v
        self.func = fn


class BinaryConstraint:
    def __init__(self, v1, v2, fn):
        self.var1 = v1
        self.var2 = v2
        self.func = fn

def allDiff(constraints, v):
    fn = lambda x, y: x != y
    for i in range(len(v)):
        for j in range(len(v)):
            if (i != j):
                constraints.append(BinaryConstraint(v[i], v[j], fn))
def setFutoshiki(caseNum):
        #set UnaryConstraint and BinaryConstraint
        uconstraints = []
        bconstraints = []

        #open file
        lines = open('testFutoshiki.txt').readlines()
        testLine = caseNum 
        l = lines[testLine]
        l = re.sub('[ ]','',l)

        n = eval(re.findall('^\d+',l)[0])
        l = re.sub('^\d+','',l)

        domain = []
        for i in range(n):
                domain.append(i+1)

        rows = []
        cols = []
        variables = dict()
        for i in range (n):
                rows.append(chr(ord('A') + i))
                cols.append(chr(ord('1') + i))
        varNames = [ x+y for x in rows for y in cols ]

        for varname in varNames:
                variables[varname] = ConstraintVar( domain,varname )

        cs=re.findall('\w+\W+\w+',l)

        for c in cs:
                if re.findall('\w+\d+<\w+\d+',c) or re.findall('\w+\d+>\w+\d+',c):
                        lvar = re.findall('^\w+\d+',c)[0]
                        rvar = re.findall('\w+\d+$',c)[0]
                        op = re.findall('\W',c)[0]
                        if op == '>':
                            fn = lambda x,y: x > y
                            fn1 = lambda x,y: x < y
                        elif op == '<':
                            fn = lambda x,y: x < y
                            fn1 = lambda x,y: x > y
                        bconstraints.append(BinaryConstraint(variables[lvar], variables[rvar], fn))
                        bconstraints.append(BinaryConstraint(variables[rvar], variables[lvar], fn1))
                else:
                        if re.findall('\w+\d+=\d+',c):
                                var = re.findall('^\w+\d+',c)[0]
                                value = re.findall('\d+$',c)[0]
                        elif re.findall('\d+=\w+\d+',c):
                                var = re.findall('\w+\d+$',c)[0]
                                value = re.findall('^\d+$',c)[0]
                        fn = functools.partial(foc, value = int(value))
                        uconstraints.append(UnaryConstraint(variables[var], fn))


        for r in rows:
            aRow = []
            for k in variables.keys():
                if (str(k).startswith(r)):
                    aRow.append(variables[k])
            allDiff(bconstraints, aRow)

        for c in cols:
            aCol = []
            for k in variables.keys():
                key = str(k)
                if (key[1] == c):
                    aCol.append(variables[k])
            allDiff(bconstraints, aCol)
        return variables, n, uconstraints, bconstraints  

def Revise(bc):
    deleted = []
    if isinstance(bc, UnaryConstraint):
        dom = list(bc.var.domain)
        for x in dom:
            if not bc.func(x):
                bc.var.domain.remove(x)
        return True
    elif isinstance(bc, BinaryConstraint):
        dom1 = list(bc.var1.domain)
        dom2 = list(bc.var2.domain)

        for x in dom1:
            needtoremove = True
            for y in dom2:
                if x == y:
                    continue
                if True == bc.func(x, y):
                    needtoremove = False
                    break
            if needtoremove:
                deleted.append(x)
                bc.var1.domain.remove(x)
        return deleted

def nodeConsistent(uc):
    domain = list(uc.var.domain)
    for x in domain:
        if not uc.func(x):
            uc.var.domain.remove(x)

def nodeConsistent(uc):
    domain = list(uc.var.domain)
    for x in domain:
        if not uc.func(x):
            uc.var.domain.remove(x)

def printDomains(vars, n):
    count = 0
    for k in sorted(vars.keys()):
        print(k, '{', vars[k].domain, '}, ', end="")
        count = count + 1
        if (0 == count % n):
            print(' ')

def printAssignment(assignment, n):
    if not assignment:
        print("Problem not consistent")
        return
    count = 0
    for k in sorted(assignment.keys()):
        print(k, '{', assignment[k], '}, ', end="")
        count += 1
        if 0 == count % n:
            print(' ')


def selectVar(assignment, variables):
        for i in variables.keys():
            if i not in assignment:
                return variables[i]
    
def backtracking(size, variables, constraints):
    for u in constraints[0]:
        nodeConsistent(u)
    return backtrack({}, size, variables, constraints)

def backtrack(assignment, size, variables, constraints):
    if len(assignment) == size*size:
        return assignment
    variable = selectVar(assignment, variables)
    for value in variable.domain:
        assignment[variable.name] = value
        inferences = FC(constraints, variables, variable, value)
        if 'inconsistent' not in inferences.keys():
            result = backtrack(assignment, size, variables, constraints)
            if result:
                return result
        assignment.pop(variable.name)
        if 'inconsistent' in inferences.keys():
            inferences.pop('inconsistent')
        for i in inferences.keys():
            variables[i].domain += inferences[i]
    return False

global counter
counter=0

def FC(constraints, variables, variable, value):
    inferences = {}
    red = []
    for i in variable.domain:
        if i != value:
            red.append(i)
    variable.domain = [value]
    inferences[variable.name] = red
    worklist = queue.Queue()
    consistent = True

    for c in constraints[1]:
        if c.var2.name == variable.name:
            worklist.put(c)
            
    while not worklist.empty():
        global counter
        counter += 1
        constraint = worklist.get()
        deleted = Revise(constraint)
        if deleted:
            for i in constraints[1]:
                if i.var2.name == constraint.var1.name:
                    worklist.put(i)
                    
            if constraint.var1.name not in inferences.keys():
                inferences[constraint.var1.name] = deleted
            else:
                inferences[constraint.var1.name] += deleted
            if not constraint.var1.domain:
                inferences['inconsistent'] = True
                consistent = False
                break
    return inferences



In [27]:
def tryFutoshiki(caseNum):
    print("<---------------------------------Board %d--------------------------------->"%(caseNum+1))
    variables, n , uconstraints, bconstraints = setFutoshiki(caseNum )
    cons = [uconstraints, bconstraints]
    t1 = time.clock()
    result = backtracking(n, variables, cons)
    t2 = time.clock()
    print("\nBacktracking with Forward Checking finished", counter, "times")
    print("Time: %.4f seconds" % (t2 - t1))
    printAssignment(result, n)
    print("\n")
    
if __name__ == '__main__':
    for i in range(4):
        tryFutoshiki(i)
        global counter
        counter = 0

<---------------------------------Board 1--------------------------------->

Backtracking with Forward Checking finished 631 times
Time: 0.0104 seconds
A1 { 2 }, A2 { 1 }, A3 { 3 }, A4 { 4 },  
B1 { 1 }, B2 { 4 }, B3 { 2 }, B4 { 3 },  
C1 { 3 }, C2 { 2 }, C3 { 4 }, C4 { 1 },  
D1 { 4 }, D2 { 3 }, D3 { 1 }, D4 { 2 },  


<---------------------------------Board 2--------------------------------->

Backtracking with Forward Checking finished 2137 times
Time: 0.0260 seconds
A1 { 5 }, A2 { 2 }, A3 { 3 }, A4 { 4 }, A5 { 1 },  
B1 { 2 }, B2 { 4 }, B3 { 1 }, B4 { 3 }, B5 { 5 },  
C1 { 1 }, C2 { 3 }, C3 { 4 }, C4 { 5 }, C5 { 2 },  
D1 { 3 }, D2 { 5 }, D3 { 2 }, D4 { 1 }, D5 { 4 },  
E1 { 4 }, E2 { 1 }, E3 { 5 }, E4 { 2 }, E5 { 3 },  


<---------------------------------Board 3--------------------------------->

Backtracking with Forward Checking finished 5643 times
Time: 0.0779 seconds
A1 { 1 }, A2 { 6 }, A3 { 2 }, A4 { 3 }, A5 { 4 }, A6 { 5 },  
B1 { 5 }, B2 { 3 }, B3 { 4 }, B4 { 1 }, B5 { 2 }